## 방문지 데이터 확인

- 결측치 및 이상치 탐색

In [86]:
import numpy as np
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
font_path = r'C:\Windows\Fonts\HMFMMUEX.TTC'
font_name = fm.FontProperties(fname=font_path).get_name()
plt.rc('font', family=font_name)
plt.rcParams['axes.unicode_minus'] = False

#### 데이터 확인

In [87]:
visit_info = pd.read_csv('data/tn_visit_area_info_방문지정보_E.csv')

In [4]:
visit_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21384 entries, 0 to 21383
Data columns (total 23 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   VISIT_AREA_ID        21384 non-null  int64  
 1   TRAVEL_ID            21384 non-null  object 
 2   VISIT_ORDER          21384 non-null  int64  
 3   VISIT_AREA_NM        21384 non-null  object 
 4   VISIT_START_YMD      21384 non-null  object 
 5   VISIT_END_YMD        21384 non-null  object 
 6   ROAD_NM_ADDR         18293 non-null  object 
 7   LOTNO_ADDR           16044 non-null  object 
 8   X_COORD              14486 non-null  float64
 9   Y_COORD              14486 non-null  float64
 10  ROAD_NM_CD           1338 non-null   float64
 11  LOTNO_CD             1338 non-null   float64
 12  POI_ID               14674 non-null  object 
 13  POI_NM               14674 non-null  object 
 14  RESIDENCE_TIME_MIN   18821 non-null  float64
 15  VISIT_AREA_TYPE_CD   21384 non-null 

In [85]:
visit_info.describe()

,VISIT_AREA_ID,VISIT_ORDER,X_COORD,Y_COORD,ROAD_NM_CD,LOTNO_CD,RESIDENCE_TIME_MIN,VISIT_AREA_TYPE_CD,VISIT_CHC_REASON_CD,LODGING_TYPE_CD,DGSTFN,REVISIT_INTENTION,RCMDTN_INTENTION,SGG_CD
count,21384,21384,14486,14486,1338,1338,18821,21384,15757,1168,15725,15725,15725,5391
mean,2306946728,6,127,37,3663097,3004636956,125,12,5,3,4,4,4,2928408544
std,1150240,5,0,0,626957,1398363995,187,7,3,3,1,1,1,1435002200
min,2304280001,1,126,34,2000002,1111017400,0,1,1,1,1,1,1,11
25%,2306050002,3,127,37,3178016,1159010700,30,6,2,1,4,3,4,1159010200
50%,2307060001,5,127,38,3350890,4111513800,60,11,4,2,4,4,4,4111313600
75%,2308190001,8,127,38,4190184,4139013450,120,21,7,5,5,5,5,4139010150
max,2309180005,38,129,38,4862065,5178025921,2280,24,11,12,5,5,5,5011025340


In [53]:
len(visit_info)

21384

In [5]:
visit_info.isnull().sum()

VISIT_AREA_ID              0
TRAVEL_ID                  0
VISIT_ORDER                0
VISIT_AREA_NM              0
VISIT_START_YMD            0
VISIT_END_YMD              0
ROAD_NM_ADDR            3091
LOTNO_ADDR              5340
X_COORD                 6898
Y_COORD                 6898
ROAD_NM_CD             20046
LOTNO_CD               20046
POI_ID                  6710
POI_NM                  6710
RESIDENCE_TIME_MIN      2563
VISIT_AREA_TYPE_CD         0
REVISIT_YN              5651
VISIT_CHC_REASON_CD     5627
LODGING_TYPE_CD        20216
DGSTFN                  5659
REVISIT_INTENTION       5659
RCMDTN_INTENTION        5659
SGG_CD                 15993
dtype: int64

In [88]:
pd.set_option('display.max_columns', None)
# temp = visit_info.head()
# temp.to_csv('visit_info_head.csv', encoding='utf-8-sig')

In [39]:
visit_info[visit_info['TRAVEL_ID'] == 'e_e000004']

,VISIT_AREA_ID,TRAVEL_ID,VISIT_ORDER,VISIT_AREA_NM,VISIT_START_YMD,VISIT_END_YMD,ROAD_NM_ADDR,LOTNO_ADDR,X_COORD,Y_COORD,ROAD_NM_CD,LOTNO_CD,POI_ID,POI_NM,RESIDENCE_TIME_MIN,VISIT_AREA_TYPE_CD,REVISIT_YN,VISIT_CHC_REASON_CD,LODGING_TYPE_CD,DGSTFN,REVISIT_INTENTION,RCMDTN_INTENTION,SGG_CD
0,2304300001,e_e000004,1,집,2023-04-30,2023-04-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21,NaN,NaN,NaN,NaN,NaN,NaN,4159011800
1,2304300002,e_e000004,2,화성 관광열차 안내소 연무대 매표소,2023-04-30,2023-04-30,경기 수원시 팔달구 창룡대로103번길 20,경기 수원시 팔달구 매향동 3-32,127,37,NaN,NaN,POI01000000ALZU7R,동대문종합시장 악세서리부자재시장,60,2,N,10,NaN,4,3,4,NaN
2,2304300003,e_e000004,3,창룡문,2023-04-30,2023-04-30,NaN,경기 수원시 팔달구 남수동,127,37,NaN,NaN,POI010000006N1USC,창룡문,30,2,N,1,NaN,4,4,4,NaN
3,2304300004,e_e000004,4,수원 화성 화홍문,2023-04-30,2023-04-30,NaN,경기 수원시 팔달구 북수동 9000-1,127,37,NaN,NaN,POI01000TR021821V,수원화성 화홍문,60,2,N,10,NaN,4,3,3,NaN
4,2304300005,e_e000004,5,집,2023-04-30,2023-05-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,390,21,NaN,NaN,NaN,NaN,NaN,NaN,4159011800


#### 1 ~ 6 여행, 방문지, 방문순서, 방문 시작일, 종료일

- 방문지역에는 다수의 여행객이 매핑

In [ ]:
temp = visit_info[['VISIT_AREA_ID', 'TRAVEL_ID', 'VISIT_ORDER', 'VISIT_AREA_NM', 'VISIT_START_YMD', 'VISIT_END_YMD']]    
temp[temp['VISIT_AREA_ID'] == 2304300002] 

,VISIT_AREA_ID,TRAVEL_ID,VISIT_ORDER,VISIT_AREA_NM,VISIT_START_YMD,VISIT_END_YMD
1,2304300002,e_e000004,2,화성 관광열차 안내소 연무대 매표소,2023-04-30,2023-04-30
6,2304300002,e_e000006,2,청량리역 경춘선,2023-04-30,2023-04-30
23,2304300002,e_e000009,8,샤오바오 고덕점,2023-04-30,2023-04-30
39,2304300002,e_e000010,12,미사리밀빛초계국수 롯데월드점,2023-04-30,2023-04-30
53,2304300002,e_e000011,11,동원 홈 푸드라운지 국립 현대미술관점,2023-04-30,2023-04-30
78,2304300002,e_e000015,11,에인절 코인 노래연습장 건대점,2023-04-30,2023-04-30
108,2304300002,e_e000020,9,가평 양떼목장,2023-04-30,2023-04-30
123,2304300002,e_e000021,11,티 카페 차희,2023-04-30,2023-04-30
142,2304300002,e_e000025,5,집,2023-04-30,2023-05-01
152,2304300002,e_e000037,4,강촌 아파트,2023-04-30,2023-05-01


- 여행id 기준으로 확인
    - 여행 (순서)여정 확인 가능 : TRAVEL_ID 기준 로우 카운트
    - 총 여행 소요일 확인 가능 : VISIT_END_YMD - VISIT_START_YMD

In [ ]:
temp = visit_info[['VISIT_AREA_ID', 'TRAVEL_ID', 'VISIT_ORDER', 'VISIT_AREA_NM', 'VISIT_START_YMD', 'VISIT_END_YMD']]    
temp = temp[temp['TRAVEL_ID']=='e_e000004'] 

In [41]:
temp

,VISIT_AREA_ID,TRAVEL_ID,VISIT_ORDER,VISIT_AREA_NM,VISIT_START_YMD,VISIT_END_YMD
0,2304300001,e_e000004,1,집,2023-04-30,2023-04-30
1,2304300002,e_e000004,2,화성 관광열차 안내소 연무대 매표소,2023-04-30,2023-04-30
2,2304300003,e_e000004,3,창룡문,2023-04-30,2023-04-30
3,2304300004,e_e000004,4,수원 화성 화홍문,2023-04-30,2023-04-30
4,2304300005,e_e000004,5,집,2023-04-30,2023-05-01


- 여행 id에 어떤 규칙이 있는지 확인해보기 

In [ ]:
# temp = visit_info[['VISIT_AREA_ID', 'TRAVEL_ID', 'VISIT_ORDER', 'VISIT_AREA_NM', 'VISIT_START_YMD', 'VISIT_END_YMD']]    
# temp = temp[temp['TRAVEL_ID']=='f_f000038'] 

#### 7 ~ 12 방문지 주소 관련 컬럼

In [ ]:
# ROAD_NM_ADDR	도로명주소
# LOTNO_ADDR	지번주소
# X_COORD	    X좌표
# Y_COORD	    Y좌표
# ROAD_NM_CD	도로명코드
# LOTNO_CD	    지번코드

- 각 주소는 방문지마다의 주소로 도로명 / 지번 / 위도, 경도 정보가 있음 
- 방문지 마다의 주소가 필요한지 확인 (여행 지역만 본다하면 주소컬럼 제외)
- 필요하면 하나의 컬럼으로 정리하자
- 기준은 도로명주소로 통일

In [ ]:
temp = visit_info[['VISIT_AREA_ID', 'TRAVEL_ID', 'VISIT_ORDER', 'VISIT_AREA_NM', 'ROAD_NM_ADDR', 'LOTNO_ADDR', 'X_COORD' ,'Y_COORD', 'ROAD_NM_CD', 'LOTNO_CD']]    

travel_id = 'f_f000038' # 'e_e000004'
temp = temp[temp['TRAVEL_ID']==travel_id] 
temp

,VISIT_AREA_ID,TRAVEL_ID,VISIT_ORDER,VISIT_AREA_NM,ROAD_NM_ADDR,LOTNO_ADDR,X_COORD,Y_COORD,ROAD_NM_CD,LOTNO_CD
21287,2304290001,f_f000038,1,숙소,영동대로 726,NaN,NaN,NaN,NaN,NaN
21288,2304290002,f_f000038,5,친척 집,엑스포아파트,NaN,NaN,NaN,NaN,NaN
21289,2304290003,f_f000038,2,친척 집,NaN,NaN,NaN,NaN,NaN,NaN
21290,2304290004,f_f000038,3,박통,NaN,NaN,NaN,NaN,NaN,NaN
21291,2304290005,f_f000038,4,김경자 소문난 대구왕뽈찜 서구점,대전 서구 월평북로 91,대전 서구 월평동 240,127.378574,36.362488,NaN,NaN
21292,2304300001,f_f000038,11,동탄 호수 자이 파밀리에 아파트,경기 화성시 동탄대로2길 19,경기 화성시 장지동 963,127.107081,37.159923,NaN,NaN
21293,2304300002,f_f000038,6,맥도날드,NaN,NaN,NaN,NaN,NaN,NaN
21294,2304300003,f_f000038,7,롯데백화점 동탄점,경기 화성시 동탄역로 160,경기 화성시 오산동 967-2419,127.097919,37.200709,NaN,NaN
21295,2304300004,f_f000038,8,동탄 호수 공원,NaN,경기 화성시 송동 54-1,127.104522,37.167834,NaN,NaN
21296,2304300005,f_f000038,9,명륜 진사갈비 화성 남동탄점,경기 화성시 동탄장지천5길 6,경기 화성시 장지동 962-6,127.106697,37.158200,NaN,NaN


- ROAD_NM_CD, LOTNO_CD 전체중 94%가 널이기도 하고 중복 정보로 제외 	

In [71]:
temp = visit_info[['VISIT_AREA_ID', 'TRAVEL_ID', 'VISIT_ORDER', 'VISIT_AREA_NM', 'ROAD_NM_ADDR', 'LOTNO_ADDR', 'X_COORD' ,'Y_COORD', 'ROAD_NM_CD', 'LOTNO_CD']]    

print(temp['ROAD_NM_CD'].isnull().sum() / len(temp))
print(temp['LOTNO_CD'].isnull().sum() / len(temp))

0.9374298540965208
0.9374298540965208


#### 13, 14 poi master 사용안하므로 제외

In [4]:
temp = visit_info[['VISIT_AREA_ID', 'TRAVEL_ID', 'VISIT_ORDER', 'VISIT_AREA_NM', 'POI_ID', 'POI_NM']]    
temp

,VISIT_AREA_ID,TRAVEL_ID,VISIT_ORDER,VISIT_AREA_NM,POI_ID,POI_NM
0,2304300001,e_e000004,1,집,NaN,NaN
1,2304300002,e_e000004,2,화성 관광열차 안내소 연무대 매표소,POI01000000ALZU7R,동대문종합시장 악세서리부자재시장
2,2304300003,e_e000004,3,창룡문,POI010000006N1USC,창룡문
3,2304300004,e_e000004,4,수원 화성 화홍문,POI01000TR021821V,수원화성 화홍문
4,2304300005,e_e000004,5,집,NaN,NaN
...,...,...,...,...,...,...
21379,2308270002,h_h003275,2,용인농촌테마파크,POI010000006S1184,용인농촌테마파크
21380,2308270003,h_h003275,3,내동마을 연꽃 단지,POI03000TR018341V,내동마을연꽃단지
21381,2308270004,h_h003275,4,외할머니 집,POI01000000CT5Q2Y,외할머니집
21382,2308270005,h_h003275,5,삼성화재 모빌리티 뮤지엄,POI03000TR018343V,삼성화재 모빌리티뮤지엄


#### 15 체류시간(분)

In [11]:
temp = visit_info[['VISIT_AREA_ID', 'TRAVEL_ID', 'VISIT_ORDER', 'VISIT_AREA_NM', 'RESIDENCE_TIME_MIN']]    

travel_id = 'e_e000004'
temp = temp[temp['TRAVEL_ID']==travel_id] 
temp

,VISIT_AREA_ID,TRAVEL_ID,VISIT_ORDER,VISIT_AREA_NM,RESIDENCE_TIME_MIN
0,2304300001,e_e000004,1,집,NaN
1,2304300002,e_e000004,2,화성 관광열차 안내소 연무대 매표소,60.0
2,2304300003,e_e000004,3,창룡문,30.0
3,2304300004,e_e000004,4,수원 화성 화홍문,60.0
4,2304300005,e_e000004,5,집,390.0


In [20]:
temp = visit_info[['VISIT_AREA_ID', 'TRAVEL_ID', 'VISIT_ORDER', 'VISIT_AREA_NM', 'RESIDENCE_TIME_MIN']]    

travel_id = 'f_f000038'
temp = temp[temp['TRAVEL_ID']==travel_id] 
temp['VISIT_ORDER'] = temp['VISIT_ORDER'].astype(int)
temp = temp.sort_values(by=['VISIT_ORDER'])

- %숙소%, %집% 의 visit_order 의 체류시간은 필요없을 듯 싶음 
- 랜드마크같은 관광지에 대한 체류시간은 서브 정보로 활용하고
- 총 소요일수가 있고, 시간까지는 굳이다 싶으면 체류시간 자체의 컬럼은 제외 고려 
- 얻을 수 있는 정보 : 여행동안의 방문지 카운트(집, 숙소제외), 여행동안의 방문지 유형별 평균체류시간 ((필요하면 / 집, 숙소제외))

In [21]:
temp

,VISIT_AREA_ID,TRAVEL_ID,VISIT_ORDER,VISIT_AREA_NM,RESIDENCE_TIME_MIN
21287,2304290001,f_f000038,1,숙소,NaN
21289,2304290003,f_f000038,2,친척 집,90.0
21290,2304290004,f_f000038,3,박통,300.0
21291,2304290005,f_f000038,4,김경자 소문난 대구왕뽈찜 서구점,60.0
21288,2304290002,f_f000038,5,친척 집,780.0
21293,2304300002,f_f000038,6,맥도날드,30.0
21294,2304300003,f_f000038,7,롯데백화점 동탄점,90.0
21295,2304300004,f_f000038,8,동탄 호수 공원,60.0
21296,2304300005,f_f000038,9,명륜 진사갈비 화성 남동탄점,120.0
21297,2304300006,f_f000038,10,브라보 노래연습장,90.0


#### 16 ~ 22 방문지, 숙소 만족도 관련 

In [89]:
def columns_eng_to_kor(df):
    mapping = {
        "VISIT_AREA_TYPE_CD": "방문지유형코드",
        "REVISIT_YN": "재방문여부",
        "VISIT_CHC_REASON_CD": "방문선택이유코드",
        "LODGING_TYPE_CD": "숙소유형코드",
        "DGSTFN": "만족도",
        "REVISIT_INTENTION": "재방문의향",
        "RCMDTN_INTENTION": "추천의향",
    }
    return df.rename(columns=mapping)


In [108]:
temp = visit_info[['VISIT_AREA_ID', 'TRAVEL_ID', 'VISIT_ORDER', 'VISIT_AREA_NM', \
                   'VISIT_AREA_TYPE_CD', 'REVISIT_YN', 'VISIT_CHC_REASON_CD', \
                    'LODGING_TYPE_CD', 'DGSTFN', 'REVISIT_INTENTION', 'RCMDTN_INTENTION']]    

df = columns_eng_to_kor(temp)

# travel_id = 'f_f000038'
# df = df[df['TRAVEL_ID']==travel_id] 
# df['VISIT_ORDER'] = df['VISIT_ORDER'].astype(int)
# df = df.sort_values(by=['VISIT_ORDER'])

In [91]:
df.head()

,VISIT_AREA_ID,TRAVEL_ID,VISIT_ORDER,VISIT_AREA_NM,방문지유형코드,재방문여부,방문선택이유코드,숙소유형코드,만족도,재방문의향,추천의향
0,2304300001,e_e000004,1,집,21,NaN,NaN,NaN,NaN,NaN,NaN
1,2304300002,e_e000004,2,화성 관광열차 안내소 연무대 매표소,2,N,10,NaN,4,3,4
2,2304300003,e_e000004,3,창룡문,2,N,1,NaN,4,4,4
3,2304300004,e_e000004,4,수원 화성 화홍문,2,N,10,NaN,4,3,3
4,2304300005,e_e000004,5,집,21,NaN,NaN,NaN,NaN,NaN,NaN


In [51]:
df.isnull().sum()

VISIT_AREA_ID        0
TRAVEL_ID            0
VISIT_ORDER          0
VISIT_AREA_NM        0
방문지유형코드              0
재방문여부             5651
방문선택이유코드          5627
숙소유형코드           20216
만족도               5659
재방문의향             5659
추천의향              5659
dtype: int64

In [111]:
# 방문지유형코드별 null값 확인
cols = ['재방문여부', '방문선택이유코드', '숙소유형코드', '만족도', '재방문의향', '추천의향']
df.groupby('방문지유형코드')[cols].apply(lambda x: x.isnull().sum())

# VIS	21	집
# VIS	22	친구/친지집
# VIS	23	사무실
# VIS	24	숙소

,재방문여부,방문선택이유코드,숙소유형코드,만족도,재방문의향,추천의향
방문지유형코드,,,,,,
1,0,0,895,0,0,0
2,0,0,1010,0,0,0
3,0,0,940,0,0,0
4,0,0,2263,0,0,0
5,0,0,179,0,0,0
6,0,0,453,0,0,0
7,0,0,598,0,0,0
8,0,0,125,0,0,0
9,0,0,1520,0,0,0


In [66]:
def overlap_count(isnull_df1, isnull_df2):
    # 겹치는 travel_id 개수
    overlap_count = len(set(isnull_df1) & set(isnull_df2))
    return overlap_count

##### 재방문여부 5651 null 인 데이터 확인

In [ ]:
isnull_df = df[df['재방문여부'].isnull() == True]
isnull_df # 대부분 방문지가 집인 곳이 null

,VISIT_AREA_ID,TRAVEL_ID,VISIT_ORDER,VISIT_AREA_NM,방문지유형코드,재방문여부,방문선택이유코드,숙소유형코드,만족도,재방문의향,추천의향
0,2304300001,e_e000004,1,집,21,NaN,NaN,NaN,NaN,NaN,NaN
4,2304300005,e_e000004,5,집,21,NaN,NaN,NaN,NaN,NaN,NaN
5,2304300001,e_e000006,1,집,21,NaN,NaN,NaN,NaN,NaN,NaN
12,2304300008,e_e000006,8,신창현 풍림아이원 1차 아파트,22,NaN,NaN,NaN,NaN,NaN,NaN
15,2305010003,e_e000006,11,집,21,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
21369,2308190004,g_g007343,4,집,21,NaN,NaN,NaN,NaN,NaN,NaN
21370,2309020001,g_g010556,1,집,21,NaN,NaN,NaN,NaN,NaN,NaN
21371,2309020002,g_g010556,8,집,21,NaN,NaN,NaN,NaN,NaN,NaN
21378,2308270001,h_h003275,1,집,21,NaN,NaN,NaN,NaN,NaN,NaN


In [54]:
isnull_df.isnull().sum()

VISIT_AREA_ID       0
TRAVEL_ID           0
VISIT_ORDER         0
VISIT_AREA_NM       0
방문지유형코드             0
재방문여부            5651
방문선택이유코드         5624
숙소유형코드           5638
만족도              5651
재방문의향            5651
추천의향             5651
dtype: int64

In [99]:
isnull_df1 = isnull_df.TRAVEL_ID.unique()
isnull_df1

array(['e_e000004', 'e_e000006', 'e_e000009', ..., 'g_g007343',
       'g_g010556', 'h_h003275'], shape=(2560,), dtype=object)

In [ ]:
# 대부분 방문지가 집인 곳이 null >> 방문지 unique값으로 재확인
visit_area_list = list(isnull_df.VISIT_AREA_NM.unique())

# 방문지유형코드로도 재확인
# VIS	1	자연관광지
# VIS	2	역사/유적/종교 시설(문화재, 박물관, 촬영지, 절 등)
# VIS	3	문화 시설(공연장, 영화관, 전시관 등)
# VIS	4	상업지구(거리, 시장, 쇼핑시설)
# VIS	5	레저/스포츠 관련 시설(스키, 카트, 수상레저)
# VIS	6	테마시설(놀이공원, 워터파크)
# VIS	7	산책로, 둘레길 등
# VIS	8	지역 축제/행사
# VIS	9	역, 터미널, 고속도로 휴게소
# VIS	10	상점
# VIS	11	식당/카페
# VIS	12	기타
# VIS	13	체험 활동 관광지
# VIS	21	집
# VIS	22	친구/친지집
# VIS	23	사무실
# VIS	24	숙소

In [102]:
visit_area_list

['집',
 '신창현 풍림아이원 1차 아파트',
 '친구 친지 집',
 '영남아파트',
 '친구 집',
 '사무실',
 '삼환아파트',
 '김해 율하시티프라디움아파트',
 '본가',
 '해솔마을 7단지 롯데캐슬 아파트',
 '하색리',
 '강촌 아파트',
 '드림하우스',
 '네이처 빌',
 '곡반정동',
 'DMC 파크뷰 자이 1단지 아파트',
 '로마 부동산',
 '동화 아파트',
 '서울대학교병원',
 '베스티안 오피스텔',
 '청계 푸르지오 시티',
 '친지 집 숙박',
 '친구 집 방문',
 '친지 집(숙박)',
 '안양 대우아파트',
 '한아름마을 라이프 현대아파트',
 '본가 집 도착',
 '산내 마을 1단지 아파트',
 '대원 레스피아 2단지 아파트',
 '부모님 집',
 '숙소',
 '자택',
 '베네치아의 아침 오피스텔',
 '광교 시티아이',
 'YG 포레스트',
 '할머니 집',
 '신흥동',
 '효성 해링턴 플레이스 아파트',
 '강동구청역 8호선',
 '부모님 댁',
 '서울숲 지구대',
 '친척 집',
 'SM 안 아주 아파트',
 '구리시 청년창업 지원센터',
 '화성 동탄 2엘에 이치 35단지 아파트',
 '삼성전자 기흥캠퍼스 정문',
 '구일 주택 ',
 '안녕동 우방아이유쉘아파트',
 '동남 패밀리 아파트',
 '수정 빌라',
 '현대 2차 아파트',
 '서울 동대문구 제기동 1140-41',
 '서울 동대문구 재기동 1140-41',
 '꿈마을 우성아파트 25동 51호',
 '아트 폴레',
 'LH 김포한강 1단지',
 '부천 아이파크 아파트',
 '딤딤섬 센트럴시티점',
 '신세계 센트럴시티',
 '부모님댁',
 '도화 현대 1차 아파트',
 '유통 상가',
 '풍림아이원 6-2단지 아파트',
 '친구의 친지 집(숙소)',
 '친구 집(숙소)',
 '예그린 하우스',
 '여자친구 집',
 '지인 집',
 '이문 1차 푸르지오 아파트',
 '친구네 집',
 '금강 프라임 빌',
 '다인로얄팰리스배곧9차오피스텔',
 '은행마을

In [100]:
visit_area_df = df[df['VISIT_AREA_NM'].apply(lambda x: x in visit_area_list)]

In [ ]:
# 방문지유형코드별 null값 확인
cols = ['재방문여부', '방문선택이유코드', '숙소유형코드', '만족도', '재방문의향', '추천의향']
visit_area_df.groupby('방문지유형코드')[cols].apply(lambda x: x.isnull().sum())

# VIS	21	집
# VIS	22	친구/친지집
# VIS	23	사무실
# VIS	24	숙소

,재방문여부,방문선택이유코드,숙소유형코드,만족도,재방문의향,추천의향
방문지유형코드,,,,,,
2,0,0,1,0,0,0
4,0,0,2,0,0,0
7,0,0,1,0,0,0
8,0,0,1,0,0,0
9,0,0,92,0,0,0
10,0,0,1,0,0,0
11,0,0,3,0,0,0
12,0,0,2,0,0,0
21,4784,4757,4777,4784,4784,4784


In [107]:
visit_area_df.groupby('방문지유형코드')[cols].apply(lambda x: x.isnull().sum()).sum()

재방문여부       5651
방문선택이유코드    5626
숙소유형코드      5803
만족도         5658
재방문의향       5658
추천의향        5658
dtype: int64

##### 방문선택이유코드 5627 null 인 데이터 확인

In [56]:
isnull_df = df[df['방문선택이유코드'].isnull() == True]
isnull_df

,VISIT_AREA_ID,TRAVEL_ID,VISIT_ORDER,VISIT_AREA_NM,방문지유형코드,재방문여부,방문선택이유코드,숙소유형코드,만족도,재방문의향,추천의향
0,2304300001,e_e000004,1,집,21,NaN,NaN,NaN,NaN,NaN,NaN
4,2304300005,e_e000004,5,집,21,NaN,NaN,NaN,NaN,NaN,NaN
5,2304300001,e_e000006,1,집,21,NaN,NaN,NaN,NaN,NaN,NaN
12,2304300008,e_e000006,8,신창현 풍림아이원 1차 아파트,22,NaN,NaN,NaN,NaN,NaN,NaN
15,2305010003,e_e000006,11,집,21,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
21369,2308190004,g_g007343,4,집,21,NaN,NaN,NaN,NaN,NaN,NaN
21370,2309020001,g_g010556,1,집,21,NaN,NaN,NaN,NaN,NaN,NaN
21371,2309020002,g_g010556,8,집,21,NaN,NaN,NaN,NaN,NaN,NaN
21378,2308270001,h_h003275,1,집,21,NaN,NaN,NaN,NaN,NaN,NaN


In [57]:
isnull_df.isnull().sum()

VISIT_AREA_ID       0
TRAVEL_ID           0
VISIT_ORDER         0
VISIT_AREA_NM       0
방문지유형코드             0
재방문여부            5624
방문선택이유코드         5627
숙소유형코드           5626
만족도              5627
재방문의향            5627
추천의향             5627
dtype: int64

In [68]:
isnull_df2 = isnull_df.TRAVEL_ID.unique()

In [69]:
overlap_count(isnull_df1, isnull_df2)

2560

In [ ]:
# 방문선택이유 활용하고자 했으나 여행객별 방문지를 확인했을때 여행객별 방문한 곳의 max는 38
# 총 unique한 visit_area_id는 1508로 컬럼으로 가져가기엔 과한것 같기에,
# EDA로 활용하고, 최종테이블로는 사용 X

# 여행객별 방문지
df.groupby('TRAVEL_ID')['VISIT_AREA_ID'].count().max()
df['VISIT_AREA_ID'].unique()

array([2304300001, 2304300002, 2304300003, ..., 2309050008, 2305280012,
       2304290011], shape=(1508,))

##### 숙소유형코드  20216 null 인 데이터 확인

In [71]:
isnull_df = df[df['숙소유형코드'].isnull() == True]
isnull_df

,VISIT_AREA_ID,TRAVEL_ID,VISIT_ORDER,VISIT_AREA_NM,방문지유형코드,재방문여부,방문선택이유코드,숙소유형코드,만족도,재방문의향,추천의향
0,2304300001,e_e000004,1,집,21,NaN,NaN,NaN,NaN,NaN,NaN
1,2304300002,e_e000004,2,화성 관광열차 안내소 연무대 매표소,2,N,10,NaN,4,3,4
2,2304300003,e_e000004,3,창룡문,2,N,1,NaN,4,4,4
3,2304300004,e_e000004,4,수원 화성 화홍문,2,N,10,NaN,4,3,3
4,2304300005,e_e000004,5,집,21,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
21379,2308270002,h_h003275,2,용인농촌테마파크,13,N,1,NaN,4,4,4
21380,2308270003,h_h003275,3,내동마을 연꽃 단지,1,N,2,NaN,3,4,3
21381,2308270004,h_h003275,4,외할머니 집,11,N,2,NaN,3,2,3
21382,2308270005,h_h003275,5,삼성화재 모빌리티 뮤지엄,13,Y,4,NaN,5,5,5


In [74]:
isnull_df.isnull().sum()

VISIT_AREA_ID        0
TRAVEL_ID            0
VISIT_ORDER          0
VISIT_AREA_NM        0
방문지유형코드              0
재방문여부             5638
방문선택이유코드          5626
숙소유형코드           20216
만족도               5645
재방문의향             5645
추천의향              5645
dtype: int64

In [75]:
isnull_df2 = isnull_df.TRAVEL_ID.unique()
overlap_count(isnull_df1, isnull_df2)

2560

##### 만족도 5659 null 인 데이터 확인

In [77]:
isnull_df = df[df['만족도'].isnull() == True]
display(isnull_df.head())
print(isnull_df.isnull().sum())

isnull_df2 = isnull_df.TRAVEL_ID.unique()
overlap_count(isnull_df1, isnull_df2)

,VISIT_AREA_ID,TRAVEL_ID,VISIT_ORDER,VISIT_AREA_NM,방문지유형코드,재방문여부,방문선택이유코드,숙소유형코드,만족도,재방문의향,추천의향
0,2304300001,e_e000004,1,집,21,NaN,NaN,NaN,NaN,NaN,NaN
4,2304300005,e_e000004,5,집,21,NaN,NaN,NaN,NaN,NaN,NaN
5,2304300001,e_e000006,1,집,21,NaN,NaN,NaN,NaN,NaN,NaN
12,2304300008,e_e000006,8,신창현 풍림아이원 1차 아파트,22,NaN,NaN,NaN,NaN,NaN,NaN
15,2305010003,e_e000006,11,집,21,NaN,NaN,NaN,NaN,NaN,NaN


VISIT_AREA_ID       0
TRAVEL_ID           0
VISIT_ORDER         0
VISIT_AREA_NM       0
방문지유형코드             0
재방문여부            5651
방문선택이유코드         5627
숙소유형코드           5645
만족도              5659
재방문의향            5659
추천의향             5659
dtype: int64


2560

##### 재방문의향 5659 null 인 데이터 확인

In [78]:
isnull_df = df[df['재방문의향'].isnull() == True]
display(isnull_df.head())
print(isnull_df.isnull().sum())

isnull_df2 = isnull_df.TRAVEL_ID.unique()
overlap_count(isnull_df1, isnull_df2)

,VISIT_AREA_ID,TRAVEL_ID,VISIT_ORDER,VISIT_AREA_NM,방문지유형코드,재방문여부,방문선택이유코드,숙소유형코드,만족도,재방문의향,추천의향
0,2304300001,e_e000004,1,집,21,NaN,NaN,NaN,NaN,NaN,NaN
4,2304300005,e_e000004,5,집,21,NaN,NaN,NaN,NaN,NaN,NaN
5,2304300001,e_e000006,1,집,21,NaN,NaN,NaN,NaN,NaN,NaN
12,2304300008,e_e000006,8,신창현 풍림아이원 1차 아파트,22,NaN,NaN,NaN,NaN,NaN,NaN
15,2305010003,e_e000006,11,집,21,NaN,NaN,NaN,NaN,NaN,NaN


VISIT_AREA_ID       0
TRAVEL_ID           0
VISIT_ORDER         0
VISIT_AREA_NM       0
방문지유형코드             0
재방문여부            5651
방문선택이유코드         5627
숙소유형코드           5645
만족도              5659
재방문의향            5659
추천의향             5659
dtype: int64


2560

##### 추천의향 5659 null 인 데이터 확인

In [79]:
isnull_df = df[df['추천의향'].isnull() == True]
display(isnull_df.head())
print(isnull_df.isnull().sum())

isnull_df2 = isnull_df.TRAVEL_ID.unique()
overlap_count(isnull_df1, isnull_df2)

,VISIT_AREA_ID,TRAVEL_ID,VISIT_ORDER,VISIT_AREA_NM,방문지유형코드,재방문여부,방문선택이유코드,숙소유형코드,만족도,재방문의향,추천의향
0,2304300001,e_e000004,1,집,21,NaN,NaN,NaN,NaN,NaN,NaN
4,2304300005,e_e000004,5,집,21,NaN,NaN,NaN,NaN,NaN,NaN
5,2304300001,e_e000006,1,집,21,NaN,NaN,NaN,NaN,NaN,NaN
12,2304300008,e_e000006,8,신창현 풍림아이원 1차 아파트,22,NaN,NaN,NaN,NaN,NaN,NaN
15,2305010003,e_e000006,11,집,21,NaN,NaN,NaN,NaN,NaN,NaN


VISIT_AREA_ID       0
TRAVEL_ID           0
VISIT_ORDER         0
VISIT_AREA_NM       0
방문지유형코드             0
재방문여부            5651
방문선택이유코드         5627
숙소유형코드           5645
만족도              5659
재방문의향            5659
추천의향             5659
dtype: int64


2560

#### 23 시군구코드

In [27]:
pd.set_option('display.float_format', '{:.0f}'.format)

In [28]:
temp = visit_info[['VISIT_AREA_ID', 'TRAVEL_ID', 'VISIT_ORDER', 'VISIT_AREA_NM', 'SGG_CD']]   

travel_id = 'f_f000038'
temp = temp[temp['TRAVEL_ID']==travel_id] 
temp['VISIT_ORDER'] = temp['VISIT_ORDER'].astype(int)
temp = temp.sort_values(by=['VISIT_ORDER'])

- 여행자의 출발지역과 방문지마다의 지역코드가 있음
- visit_order의 여행지 주활동지역을 알 수 있는 정보로 다른 정보로 대체가능하다면 제외 고려

In [29]:
temp

,VISIT_AREA_ID,TRAVEL_ID,VISIT_ORDER,VISIT_AREA_NM,SGG_CD
21287,2304290001,f_f000038,1,숙소,1168010400
21289,2304290003,f_f000038,2,친척 집,3011011800
21290,2304290004,f_f000038,3,박통,3017011300
21291,2304290005,f_f000038,4,김경자 소문난 대구왕뽈찜 서구점,NaN
21288,2304290002,f_f000038,5,친척 집,3020014100
21293,2304300002,f_f000038,6,맥도날드,3023012600
21294,2304300003,f_f000038,7,롯데백화점 동탄점,NaN
21295,2304300004,f_f000038,8,동탄 호수 공원,NaN
21296,2304300005,f_f000038,9,명륜 진사갈비 화성 남동탄점,NaN
21297,2304300006,f_f000038,10,브라보 노래연습장,4159013500


#### (After) 데이터 정제 및 데이터 가공

- 삭제할 컬럼 = ['ROAD_NM_ADDR','LOTNO_ADDR','X_COORD','Y_COORD','ROAD_NM_CD','LOTNO_CD','POI_ID','POI_NM','RESIDENCE_TIME_MIN','VISIT_CHC_REASON_CD','LODGING_TYPE_CD','SGG_CD']
- 방문지가 VIS={집:21, 친구/친지집:22, 사무실:23, 숙소:24} 코드에 해당하는 곳이 null임을 확인
- 여행id별 20~22 컬럼에 대하여 총 방문지들의 대체적인 만족도 평가로써 평균값(집 제외) 사용
- 재방문여부는 여행동안의 방문지중 재방문여부가 얼마나 되는지 비율 확인해서 활용 고려
- 집계시 VIS={집:21, 친구/친지집:22, 사무실:23, 숙소:24} 는 제외